In [3]:
import pandas as pd

df = pd.read_csv('/rutland_df.csv')

In [5]:
for date in range(len(df['datetime'])):
        datetime =  df['datetime'][date]
        year = str(datetime[:4])
        month = str(datetime[5:7])
        day = str(datetime[8:10])
        hour = str(datetime[11:13])
        final_datetime = year + month + day + hour
        df['datetime'][date] = final_datetime
df['datetime'] = pd.to_datetime(df['datetime'], format='%Y%m%d%H')
print('datetimes converted')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


datetimes converted


In [6]:
df = df.drop(['Unnamed: 0'], axis=1)
df = df.drop(['0'], axis=1)
df = df.reset_index(drop=True)
df = df.set_index(['datetime'])

In [8]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import TimeseriesGenerator

df_input = df[['lmp','loadMW','temp','visibility','wspd','cloudcover','precip','pressure', 'humidity']]

print(df_input.head())
print(df_input.describe())
print(df_input.query('lmp < 0.1'))
print(df_input.info())
scalar = MinMaxScaler(feature_range=(0,1))
    # transforms into numpy array
scalar = scalar.fit(df_input)
data_normalized = scalar.transform(df_input)
print(data_normalized)

featMat = data_normalized
targetMat = data_normalized[:, 0]  #[:, :1]

   
print(targetMat)

x_train, x_test, y_train, y_test = train_test_split(featMat, targetMat, test_size=0.3, random_state=42, shuffle=False)
print(x_train.shape)
print (x_test.shape)

train_gen = TimeseriesGenerator(x_train, y_train, length=24, sampling_rate=1, batch_size=64)
test_gen = TimeseriesGenerator(x_test, y_test, length=24, sampling_rate=1, batch_size=64)


                       lmp   loadMW  temp  visibility  wspd  cloudcover  \
datetime                                                                  
2015-05-01 00:00:00  19.88  10180.0  51.7         9.9  12.2        48.7   
2015-05-01 01:00:00  16.35   9790.0  51.7         9.9  14.5        98.2   
2015-05-01 02:00:00  15.14   9550.0  48.3         9.9  12.3        98.2   
2015-05-01 03:00:00  14.74   9520.0  48.2         9.9   8.9        98.2   
2015-05-01 04:00:00  16.04   9790.0  48.2         9.9   5.7        98.2   

                     precip  pressure  humidity  
datetime                                         
2015-05-01 00:00:00     0.0    1014.6     62.46  
2015-05-01 01:00:00     0.0    1015.0     58.28  
2015-05-01 02:00:00     0.0    1014.9     66.19  
2015-05-01 03:00:00     0.0    1014.5     54.17  
2015-05-01 04:00:00     0.0    1014.5     66.43  
                lmp        loadMW          temp    visibility          wspd  \
count  56856.000000  56856.000000  56856.0000

In [9]:
import tensorflow as tf
import numpy as np
from keras.layers import LSTM, LeakyReLU, Dropout, Input, Dense, Conv1D
from keras.models import Model

inputs = Input(shape=(24, 9))
    #model = tf.keras.Sequential()
conv1 = Conv1D(128, kernel_size=10, activation='ReLU', strides=1)(inputs)
convDrop = Dropout(0.2)(conv1)

conv2 = Conv1D(64, kernel_size=9, activation='ReLU', strides=1)(convDrop)
convDrop = Dropout(0.25)(conv2)

Lstm = LSTM(64, activation='tanh', return_sequences=True)(convDrop)
    #model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
Lstm = LSTM(64, activation='tanh', return_sequences=True)(Lstm)
lstmDrop = Dropout(0.3)(Lstm)
    #dense = Dense(32, activation=LeakyReLU(alpha=0.1))(lstmDrop)
outputs = Dense(1, activation='linear')(lstmDrop)
model = Model(inputs, outputs)
    #model.add(tf.keras.layers.LSTM(64, return_sequences=True))
    #model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
    ##model.add(tf.keras.layers.Dropout(0.4))
    #model.add(tf.keras.layers.Dense(9))

print(model.summary())
    
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                        patience=2,
                                                        mode='min')
model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(learning_rate=0.000001, clipvalue=0),
                metrics=[tf.metrics.MeanAbsoluteError()])

history = model.fit_generator(train_gen, validation_data=test_gen, epochs=50, shuffle=False)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 24, 9)]           0         
                                                                 
 conv1d (Conv1D)             (None, 15, 128)           11648     
                                                                 
 dropout (Dropout)           (None, 15, 128)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 7, 64)             73792     
                                                                 
 dropout_1 (Dropout)         (None, 7, 64)             0         
                                                                 
 lstm (LSTM)                 (None, 7, 64)             33024     
                                                                 
 lstm_1 (LSTM)               (None, 7, 64)             33024 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


622/622 [==============================] - 26s 34ms/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 2/50
622/622 [==============================] - 23s 37ms/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 3/50
622/622 [==============================] - 21s 34ms/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 4/50
622/622 [==============================] - 22s 35ms/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 5/50
622/622 [==============================] - 21s 33ms/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 6/50
622/622 [==============================] - 22s 35ms/step - loss: nan - mean_absolute_error: nan - val_loss: nan - val_mean_absolute_error: nan
Epoch 7/50
622/622 [==============================] - 21s 33ms/step - loss: nan - mean_